## TPC2: Conversor de MarkDown para HTML

Criar em Python um pequeno conversor de MarkDown para HTML para os elementos descritos na "Basic Syntax" da Cheat Sheet:

In [5]:
import re

### Cabeçalhos: linhas iniciadas por "# texto", ou "## texto" ou "### texto"

In: `# Exemplo`

Out: `<h1>Exemplo</h1>`

In [12]:
exemplo1 = "# alunoX"
exemplo2 = "## tpc"
exemplo3 = "### PLC"
exemploextra = "## tentativa."
casoparagem = "ola"

def conversor_cabecalho(texto):
    #inicio = #{1,3} , seguido de string \s + "possível(.)
    tipo = re.compile(r"^(#{1,3})\s+(.+)")
    m = tipo.match(texto)
    if not m:
        return texto
    nivel = len(m.group(1))
    mensagem = m.group(2)

    return f"<h{nivel}>{mensagem}</h{nivel}>"

print(conversor_cabecalho(exemplo1))
print(conversor_cabecalho(exemplo2))
print(conversor_cabecalho(exemplo3))
print(conversor_cabecalho(exemploextra))
print(conversor_cabecalho(casoparagem))

<h1>alunoX</h1>
<h2>tpc</h2>
<h3>PLC</h3>
<h2>tentativa.</h2>
ola


### Bold: pedaços de texto entre "**":

In: `Este é um **exemplo** ...`

Out: `Este é um <b>exemplo</b> ...`

### Itálico: pedaços de texto entre "*":

In: `Este é um *exemplo* ...`

Out: `Este é um <i>exemplo</i> ...`

In [13]:
exemplo1 = "Este é um **exemplo**"
exemplo2 = "Este é um *exemplo*"
exemplo3 = "Este é um exemplo"
exemplo4 = "Este é um ***exemplo***"

def bold_italico(texto):
    texto = re.sub(r"\*\*\*(.+?)\*\*\*", r"<b><i>\1</i></b>", texto)
    texto = re.sub(r"\*\*(.+?)\*\*", r"<b>\1</b>", texto)
    texto = re.sub(r"\*(.+?)\*", r"<i>\1</i>", texto)
    return texto

print(bold_italico(exemplo1))
print(bold_italico(exemplo2))
print(bold_italico(exemplo3))
print(bold_italico(exemplo4))

Este é um <b>exemplo</b>
Este é um <i>exemplo</i>
Este é um exemplo
Este é um <b><i>exemplo</i></b>


### Lista numerada:

In:
```
1. Primeiro item
2. Segundo item
3. Terceiro item
```

Out:
```
<ol>
<li>Primeiro item</li>
<li>Segundo item</li>
<li>Terceiro item</li>
</ol>
```

In [14]:
import re

exemplo = """1. Primeiro item
2. Segundo item
3. Terceiro item"""

def lista_numerada(lista):
    linhas = lista.splitlines()
    output = []
    na_lista = False
    itens = []

    padrao = re.compile(r'^\s*\d+\.\s+(.*)$')

    for linha in linhas:
        m = padrao.match(linha)
        if m:
            conteudo = m.group(1)
            if not na_lista:
                na_lista = True
                itens = []
            itens.append(conteudo)
        else:
            if na_lista:
                output.append("<ol>")
                for item in itens:
                    output.append(f"<li>{item}</li>")
                output.append("</ol>") 
                na_lista = False
                itens = []
            output.append(linha) 

    # fechar lista se acabou dentro de uma lista
    if na_lista:
        output.append("<ol>")
        for item in itens:
            output.append(f"<li>{item}</li>")
        output.append("</ol>")

    return "\n".join(output)

print(lista_numerada(exemplo))


<ol>
<li>Primeiro item</li>
<li>Segundo item</li>
<li>Terceiro item</li>
</ol>


### Link: [texto](endereço URL)

In: `Como pode ser consultado em [página da UC](http://www.uc.pt)`

Out: `Como pode ser consultado em <a href="http://www.uc.pt">página da UC</a>`

In [15]:
exemplo = "Como pode ser consultado em [página da UC](http://www.uc.pt)"

def texto_link(link):
    texto = re.compile(r"\[([^\]]+)\]\(([^)]+)\)")
    #Lambda m usado para não criar outra função
    return texto.sub(lambda m: f'<a href="{m.group(2)}">{m.group(1)}</a>', link)

print(exemplo)
print(texto_link(exemplo))


Como pode ser consultado em [página da UC](http://www.uc.pt)
Como pode ser consultado em <a href="http://www.uc.pt">página da UC</a>


### Imagem: ![texto alternativo](path para a imagem)

In: `Como se vê na imagem seguinte: ![imagem dum coelho](http://www.coellho.com) ...`

Out: `Como se vê na imagem seguinte: <img src="http://www.coellho.com" alt="imagem dum coelho"/> ...`

In [16]:
exemplo = r"Como se vê na imagem seguinte: ![imagem dum coelho](http://www.coellho.com) ..."

def mudar_imagem(linha):
    regex = re.compile(r"!\[([^\]]+)\]\(([^)]+)\)")
    return regex.sub(lambda m: f'<img src="{m.group(2)}" alt="{m.group(1)}"/>', linha)

print(exemplo)
print(mudar_imagem(exemplo))


Como se vê na imagem seguinte: ![imagem dum coelho](http://www.coellho.com) ...
Como se vê na imagem seguinte: <img src="http://www.coellho.com" alt="imagem dum coelho"/> ...


# Conversor

In [17]:
texto = """## Título
Texto em **negrito** e *itálico*
1. Primeiro Item
2. Segundo Item
Aqui temos um [link](http://teste.com)
Aqui temos uma imagen ![alt](http://img.com/x.png)"""

def conversor_markdown_html(texto):
    # lista de transformações
    transformacoes = [
        conversor_cabecalho,
        bold_italico,
        mudar_imagem,
        texto_link
    ]

    # aplicar transformações linha a linha
    linhas = []
    for linha in texto.splitlines():
        for func in transformacoes:
            linha = func(linha)
        linhas.append(linha)

    # juntar linhas
    texto = "\n".join(linhas)
    texto = lista_numerada(texto)

    return texto

print(conversor_markdown_html(texto))


<h2>Título</h2>
Texto em <b>negrito</b> e <i>itálico</i>
<ol>
<li>Primeiro Item</li>
<li>Segundo Item</li>
</ol>
Aqui temos um <a href="http://teste.com">link</a>
Aqui temos uma imagen <img src="http://img.com/x.png" alt="alt"/>
